In [1]:
%matplotlib qt5 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from fastdtw import fastdtw

In [2]:
data = pd.read_csv("train.csv")
filtered_data = data[data["sign"] == "puzzle"]
csv_path = "filter_for_word.csv"
filtered_data.to_csv(csv_path, index=False)

In [3]:
"""One file path""" 
data_total = np.zeros((1, 543, 8))

def extract_descriptors(data_train):
    data_train_descriptor = np.zeros((1, 543, 8))
    for i in range(0,543):
        d1 = np.mean(data_train[i, :, 0])
        d2 = np.mean(data_train[i, :, 1])
        d3 = np.max(data_train[i, :, 0])
        d4 = np.max(data_train[i, :, 1])
        d5 = np.min(data_train[i, :, 0])
        d6 = np.min(data_train[i, :, 1])
        d7 = np.sum(np.square(data_train[i, :, 0]))
        d8 = np.sum(np.square(data_train[i, :, 1]))
        data_train_descriptor[:, i, :] = [d1, d2, d3, d4, d5, d6, d7, d8]
    return data_train_descriptor
    
for i in range(filtered_data.shape[0]):
    path_one_file = filtered_data.iloc[i]["path"]
    test = pd.read_parquet(path_one_file)
    cleaned_data = test[["frame", "landmark_index", "x", "y"]]
    num_samples = cleaned_data["frame"].nunique()
    cleaned_data = cleaned_data[["frame", "landmark_index", "x", "y"]].to_numpy()
    cleaned_data = cleaned_data.reshape(543, num_samples, 4)

    data_with_descriptors = extract_descriptors(cleaned_data)
    data_total = np.append(data_total, data_with_descriptors, axis=0)

In [4]:
# def extract_testing_descriptor(data_train, num_samples):
#     data_testing_descriptor = np.zeros((1, 543, 2))
#     for i in num_samples:
#         for i in range(0,543):  
#     return data_train_descriptor

def process_path_data(path_number, filtered_data):
    path_file = filtered_data.iloc[path_number]["path"]
    data = pd.read_parquet(path_file)
    cleaned_data = data[["frame", "x", "y"]]
    num_samples = cleaned_data["frame"].nunique()
    cleaned_data = cleaned_data[["x", "y"]].to_numpy()
    seq = cleaned_data.reshape(num_samples, 543, 2)
    return seq

seq1 = process_path_data(1, filtered_data)
seq2 = process_path_data(4, filtered_data)

In [5]:
# Interpolar ambas secuencias a una longitud común (por ejemplo, 50 fotogramas)
num_frames = 30
seq1_interp = np.zeros((num_frames, 543, 2))
seq2_interp = np.zeros((num_frames, 543, 2))

for i in range(543):
    for j in range(2):
        f1 = interp1d(np.arange(seq1.shape[0]), seq1[:, i, j], kind='linear')
        f2 = interp1d(np.arange(seq2.shape[0]), seq2[:, i, j], kind='linear')

        seq1_interp[:, i, j] = f1(np.linspace(0, seq1.shape[0]-1, num_frames))
        seq2_interp[:, i, j] = f2(np.linspace(0, seq2.shape[0]-1, num_frames))

print(seq1_interp.shape)
print(seq2_interp.shape)

plt.figure()
plt.plot(seq1_interp[:, 0, 0], seq1_interp[:, 0, 1], 'r')
plt.plot(seq2_interp[:, 0, 0], seq2_interp[:, 0, 1], 'b')

# total_distance = 0

# for i in range(543):
#     distance, _ = fastdtw(seq1_interp[:, i, :], seq2_interp[:, i, :])
#     total_distance += distance


(30, 543, 2)
(30, 543, 2)


In [6]:
""" visualizar datos"""

data_descriptors_rendering = np.load("data_with_descriptors_puzzle.npy")

plt.plot(data_descriptors_rendering[1, :, 0], data_descriptors_rendering[1, :, 1], 'o')
